<a href="https://colab.research.google.com/github/Geo-99/DL_project_24_HR_GS/blob/main/2_rollout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install owslib
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
import os
import geopandas as gpd
from shapely.geometry import Polygon
import time
from owslib.wms import WebMapService
from ultralytics import YOLO
import pandas as pd

In [ ]:
os.chdir('/content/drive/MyDrive/XY')

In [ ]:
berlin_points_gpkg = "./3_rollout_berlin/berlin_points.gpkg"
dop20_ber_wms = "https://gdi.berlin.de/services/wms/truedop_2024?REQUEST=GetCapabilities&SERVICE=wms"
yolo_model = "./2_cls/1_m_model/weights/best.pt"

In [ ]:
berlin_points_join = gpd.read_file(berlin_points_gpkg)
berlin_points_join["constr"] = 0
berlin_points_join["constr_conf"] = 0.0

In [ ]:
# size of the tiles in meters
box_size = 96
# Define output image size
output_size = (box_size / 0.2, box_size / 0.2)
# as integer
output_size = (int(output_size[0]), int(output_size[1]))
# Define the output image format
output_format = "image/png"

# Create tiles gpkg
tiles = gpd.GeoDataFrame(columns=["geometry", "Gemeinde_n", "Gemeinde_s", "constr", "constr_conf", "original_id"], crs="EPSG:32633")

# Source CRS & Target CRS
source_crs = "EPSG:32633"
target_crs = "EPSG:3857"

In [ ]:
# Connect to the WMS service
wms = WebMapService(dop20_ber_wms)
# Define layer name
layer_name = "truedop_2024"

In [ ]:
# autosave (because of stops/disconnects in between)
save_list = [i for i in range(1000, 100001, 1000)]
save_path_points = "./points_autosave.gpkg"
save_path_tiles = "./tiles_autosave.gpkg"

In [ ]:
# Run over whole dataset
cont = range(len(berlin_points_join))

# Continue points where it stopped last time
#cont_start = 60995
#cont = range(cont_start, len(berlin_points_join))
#cont = range(cont_start, 70050)

In [ ]:
# Load YOLO model
model = YOLO(yolo_model)

# start timer
start = time.time()

#for i in range(len(berlin_points_join)):
for i in cont:
    # Print the progress
    progress = i + 1
    print(f"\n\nProgress: {progress}/{len(berlin_points_join)}")
    # Print current time
    print(f"Time: {(time.time() - start) / 60} minutes\n")
    # Calculate remaining time
    print(f"Remaining time:\n"
          f"{(((time.time() - start) / progress) * (len(berlin_points_join) - progress)) / 60} minutes\n"
          f"{(((time.time() - start) / progress) * (len(berlin_points_join) - progress)) / 3600} hours\n\n\n")

    point = berlin_points_join.iloc[i].geometry
    district_s = int(berlin_points_join.iloc[i].Gemeinde_s)
    district_n = berlin_points_join.iloc[i].Gemeinde_n
    x, y = point.x, point.y
    x_min, y_min, x_max, y_max = x - box_size/2, y - box_size/2, x + box_size/2, y + box_size/2
    # Create a polygon from x_min, y_min, x_max, y_max
    polygon_geom = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)])
    polygon = gpd.GeoDataFrame({"geometry": [polygon_geom], "Gemeinde_n": district_n, "Gemeinde_s": district_s,
                                "constr": 0, "constr_conf": 0.0, "original_id": i+1},
                                crs="EPSG:32633")
    # Append polygon to tiles
    tiles = pd.concat([tiles, polygon], ignore_index=True)
    # Create 4 points for the box
    points = gpd.points_from_xy([x_min, x_max, x_max, x_min], [y_max, y_max, y_min, y_min], crs=source_crs)
    # Points in new CRS
    points_new = points.to_crs(target_crs)

    # Get the new x_min, y_min, x_max, y_max
    x_min_new, y_min_new, x_max_new, y_max_new = points_new.x[0], points_new.y[3], points_new.x[2], points_new.y[1]

    # Get the image from the WMS
    img = wms.getmap(layers=[layer_name],
                     srs=target_crs,
                     bbox=[x_min_new, y_min_new, x_max_new, y_max_new],
                     size=output_size,
                     format=output_format
                )

    # Save the image
    path_image = f"./images/{i}_{len(berlin_points_join)}.png"
    with open(path_image, "wb") as out:
        out.write(img.read())

    # YOLO
    pred = model(path_image)
    pred_class = pred[0].probs.top1
    conf = pred[0].probs.data
    conf_constr = round(conf[0].item(), 2)

    if pred_class == 0:
        berlin_points_join.at[i, "constr"] = 1
        tiles.at[len(tiles)-1, "constr"] = 1

    berlin_points_join.at[i, "constr_conf"] = conf_constr
    tiles.at[len(tiles)-1, "constr_conf"] = conf_constr

    # Delete the current image
    os.remove(path_image)

    # autosave
    if i in save_list:
      if os.path.exists(save_path_points):
        os.remove(save_path_points)
      if os.path.exists(save_path_tiles):
        os.remove(save_path_tiles)
      tiles.to_file(save_path_tiles, driver="GPKG")
      berlin_points_join.to_file(save_path_points, driver="GPKG")

# Save the tiles & points
tiles.to_file("./tiles.gpkg", driver="GPKG")
berlin_points_join.to_file("./points.gpkg", driver="GPKG")



Progress: 60996/96633
Time: 1.2997786204020182e-05 minutes

Remaining time:
8.088477330642792e-06 minutes
1.3666524077607317e-07 hours





<ipython-input-11-eb6034404f7e>:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tiles = pd.concat([tiles, polygon], ignore_index=True)


Streaming output truncated to the last 5000 lines.



image 1/1 /content/drive/MyDrive/HG_Constr_YOLO_Class/berlin_rollout/images/62700_96633.png: 480x480 constr 0.60, no_constr 0.40, 387.2ms
Speed: 4.6ms preprocess, 387.2ms inference, 0.1ms postprocess per image at shape (1, 3, 480, 480)


Progress: 62702/96633
Time: 68.59192750453948 minutes

Remaining time:
37.118321969111065 minutes
0.6186387018863806 hours




image 1/1 /content/drive/MyDrive/HG_Constr_YOLO_Class/berlin_rollout/images/62701_96633.png: 480x480 no_constr 0.99, constr 0.01, 380.2ms
Speed: 4.7ms preprocess, 380.2ms inference, 0.1ms postprocess per image at shape (1, 3, 480, 480)


Progress: 62703/96633
Time: 68.62664370934168 minutes

Remaining time:
37.13542080457675 minutes
0.6189236822265066 hours




image 1/1 /content/drive/MyDrive/HG_Constr_YOLO_Class/berlin_rollout/images/62702_96633.png: 480x480 constr 0.72, no_constr 0.28, 636.1ms
Speed: 6.1ms preprocess, 636.1ms inference, 0.1ms postprocess per image at shap

HTTPError: 502 Server Error: Proxy Error for url: https://gdi.berlin.de/services/wms/truedop_2024?SERVICE=WMS&&service=WMS&version=1.1.1&request=GetMap&layers=truedop_2024&styles=&width=480&height=480&srs=EPSG%3A3857&bbox=1514779.7841908634%2C6874257.591639206%2C1514939.8632771936%2C6874418.072201811&format=image%2Fpng&transparent=FALSE&exceptions=application%2Fvnd.ogc.se_xml&bgcolor=0xFFFFFF